# GR 3. - analiza MNIST database

In [1]:
# 1. Wprowadzenie i wstępna analiza danych.

Format każdego rekordu: 
    X - dwuwymiarowy numpy array zawierający kolor poszczególnych pikseli w skali szarości
    Y - cyfra reprezentowana przez dany rekord
    
https://en.wikipedia.org/wiki/MNIST_database
https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/
https://www.tensorflow.org/datasets/catalog/mnist
https://towardsdatascience.com/image-classification-in-10-minutes-with-mnist-dataset-54c35b77a38d
    
Plan działania: 
    10.01 - 16.01 - wstępny zarys i weryfikacja zaproponowanych rozwiązań
    20.01 - jak najszybciej - ukończenie prac


In [ ]:
# 2. Klasyfikatory z zajęć - wybór najlepszego:

    Metoda najbliżego sąsiada
    Metoda k-najbliższych sąsiadów
    Metoda najbliższego prototypu
    Naiwny klasyfikatory Bayesa
    Drzewa decyzyjne

Przerabiamy na ramkę danych?

In [ ]:
# 3. Preprocessing obrazów dla najlepszego klasyfikatora

In [ ]:
# 4. Ponowna weryfikacja klasyfikatorów dla najbardziej obiecujących preprocessingów

In [ ]:
# 5. Próba użycia sieci neuronowych.

In [ ]:
# 6. Porównanie rozwiązań i wnioski.

In [ ]:
# 7. Próba sklasyfikowania małego zbioru cyfr napisanych przez nas odręcznie